In [2]:
import numpy as np
from PIL import Image
import math

In [6]:
def select(data, R, G, B):
    data_copy = data.copy()
    data_copy[:, :, 2] = np.select([B], [data_copy[:, :, 2]])
    data_copy[:, :, 1] = np.select([G], [data_copy[:, :, 1]])
    data_copy[:, :, 0] = np.select([R], [data_copy[:, :, 0]])
    data_copy[np.any(data_copy[:, :] == 0, axis=2)] = [0, 0, 0]
    return data_copy


def distance(data, ligne, colonne):
    ligne_c, colonne_c, _ = data.shape
    ligne_c = ligne_c // 2
    colonne_c = colonne_c // 2
    return ((ligne_c - ligne) ** 2 + (colonne_c - colonne) ** 2) ** (.5)


def detectBalise(image):
    image = image.resize((160,300),Image.ANTIALIAS)
    data = np.asarray(image)

    R = np.logical_and(10 < data[:, :, 0], data[:, :, 0] < 35)
    G = np.logical_and(80 < data[:, :, 1], data[:, :, 1] < 100)
    B = np.logical_and(160 < data[:, :, 2], data[:, :, 2] < 180)

    image_blue = select(data, R, G, B)
    R = np.logical_and(40 < data[:, :, 0], data[:, :, 0] < 65)
    G = np.logical_and(130 < data[:, :, 1], data[:, :, 1] < 145)
    B = np.logical_and(100 < data[:, :, 2], data[:, :, 2] < 120)

    image_green = select(data, R, G, B)
    R = np.logical_and(180 < data[:, :, 0], data[:, :, 0] < 190)
    G = np.logical_and(60 < data[:, :, 1], data[:, :, 1] < 75)
    B = np.logical_and(65 < data[:, :, 2], data[:, :, 2] < 75)

    image_red = select(data, R, G, B)
    lignes_b, colonnes_b = np.where(image_blue[:, :, 0] != 0)
    lignes_g, colonnes_g = np.where(image_green[:, :, 0] != 0)
    lignes_r, colonnes_r = np.where(image_red[:, :, 0] != 0)

    # balise on image
    avg_c_r = np.average(colonnes_r)
    avg_c_b = np.average(colonnes_b)
    avg_c_g = np.average(colonnes_g)
    avg_l_g = np.average(lignes_g)
    avg_l_b = np.average(lignes_b)
    avg_l_r = np.average(lignes_r)
    balise = avg_c_r <= avg_c_b and (
        np.min(lignes_b) <= avg_l_r <= np.max(lignes_b))
    balise = balise and (np.min(colonnes_b) <= avg_c_g <=
                         np.max(colonnes_b)) and avg_l_g <= avg_l_b

    #balise in front
    baliseInFront = False
    itIsLeft = None
    pourcentage = None

    if balise:
        colonne_c = data.shape[1] // 2
        dist_corner = distance(data, 0, 0)
        dist_r = .2 <= (distance(data, avg_l_r, avg_c_r) / dist_corner) <= .6
        dist_g = .2 <= (distance(data, avg_l_g, avg_c_g) / dist_corner) <= .6
        dist_b = .2 <= (distance(data, avg_l_b, avg_c_b) / dist_corner) <= .6
        baliseInFront = balise and dist_r and dist_b and dist_g
        baliseInFront = baliseInFront and avg_c_b > colonne_c and avg_c_r < colonne_c and avg_c_g > colonne_c

        if not baliseInFront:
            itIsLeft = avg_c_b < colonne_c
            pourcentage = abs((avg_c_b + avg_c_r) / 2. -
                              colonne_c) / colonne_c

    return balise, baliseInFront, itIsLeft, pourcentage

# 3-uplet
# 0 - is balise on image capture ?
# 1 - is balise in front of robot ?
# 2 - if balise on image, however not in front say if it is on more on left (True) or more on right (False)
# 3 - pourcentage is how many the balise is from the center image.
detectBalise(Image.open("./balise_right.jpg"))

(True, False, False, 0.5868174287307522)